In [ ]:
from flask import Flask, request, abort
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import *

app = Flask(__name__)

# 獲取存取通道的令牌TOKEN
line_bot_api = LineBotApi('YOUR_CHANNEL_ACCESS_TOKEN')
# 使用加密處理接收消息
handler = WebhookHandler('YOUR_CHANNEL_SECRET')

# 監聽所有來自 /callback 的 Post Request
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

'''
reply_message 規定只能在回復一次，也就是說每收到一則訊息只能回覆一次，
如果想要回覆多條訊息的話，可以使用 push_message，
Free的方案是 不提供 push_message 的功能，
需要將方案改成 developer trial 才能用 push_message
'''

# 處理訊息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    # 可以修改(text = 回應的文字)
    message = TextSendMessage(text=event.message.text)
    #message = TestSendMessage(text="Hello World"
    line_bot_api.reply_message(event.reply_token, message)

import os
if __name__ == "__main__":
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port)
